In [28]:

###############################################################
#  This code is part of project Spinux developed at Spine Labs
#  Dept. of BME,
#  IIT Hyderabad
#  www.iith.ac.in/~mohanr
#
#  Revision History
#  -----------------------
#   Name             Date            Changes
#   ---------       -------     ----------------------
#   madhav vinodh  dd/mm/yy      Initial Creation
#   raghu S                      clean-up

###############################################
# This file is invoked by scripts/gen/3_mapping_json.py
#
# This file parses the xls file Define/Network/motor_ckts.xlsx
# The xls file contians a motif for creation of cell groups
# and synaptic connections.
# cell_groups.xlsx and nc_explicit.xlsc are created using this info
###############################################

#this file generates the spinal cord circuitry basded on muscle synergies/ movement primitives and spinal cord elementary circuit theory.
#see details in manuscript for "elementary circuit theory"


import os
import json
import pandas as pd
cwd = os.getcwd()
cwd = cwd 

#fns for import files
def get_movements_input_file():
    return cwd + '/movement_types.xlsx'
def get_muscles_input_file():
    return cwd + '/muscles.xlsx'
def get_spinal_neurons_input_file():
    return cwd + '/spinal_neurons.xlsx'
# def get_neuron_props_input_file():
#     return cwd + '/neuron_props.xlsx'
def get_connection_rules_input_file():
    return cwd + '/connection_rules.xlsx'

#fns for export output files
def get_nc_explicit_file():
    return cwd + '/nc_explicit.xlsx'
def get_cellgroups_file():
    return cwd + '/cell_groups.xlsx'







########################################################################################################
#fnx to read xlsx sheets 
#load movement_types and append them to movements_roster
#load_movements = pd.read_excel(get_movements_input_file(),sheetname='movement_types')
movements_roster = pd.read_excel(get_movements_input_file(),sheetname='movement_types')

muscles_roster = pd.read_excel(get_muscles_input_file(),sheetname='muscles')

neurons_roster = {}
neurons_roster['motoneurons'] = []
neurons_roster['interneurons'] = []
neurons_roster['drg_neurons'] = []
load_motoneurons = pd.read_excel(get_spinal_neurons_input_file(),sheetname='motoneurons')
for a in range(0,len(load_motoneurons)):
    tempdf = load_motoneurons.iloc[a].to_dict() 
    neurons_roster['motoneurons'].append(tempdf)

load_interneurons = pd.read_excel(get_spinal_neurons_input_file(),sheetname='interneurons')
for a in range(0,len(load_interneurons)):
    tempdf = load_interneurons.iloc[a].to_dict() 
    neurons_roster['interneurons'].append(tempdf)

load_drg_neurons = pd.read_excel(get_spinal_neurons_input_file(),sheetname='drg_neurons')
for a in range(0,len(load_drg_neurons)):
    tempdf = load_drg_neurons.iloc[a].to_dict() 
    neurons_roster['drg_neurons'].append(tempdf)

#load connection_rules and append them to connections_rosters

connections_roster = []
load_connection_rules = pd.read_excel(get_connection_rules_input_file(),sheetname='connection_rules')
connections = list(load_connection_rules["connection_type"])
for a in range(0,len(connections)):
    tempdf = load_connection_rules.iloc[a].to_dict() 
    connections_roster.append(tempdf)

connections_info = {}
connections_info["self"] = []
connections_info["agonist"] = []
connections_info["antagonist"] = []
for coninfo in connections_roster:
    for mgrp in coninfo['target_muscle_group'].split(","):
        #print mgrp
        cinfo = { 'src': coninfo['source'], 'dest': coninfo['target'], 'props':eval(coninfo['props']), 'side':coninfo['side'] }
        connections_info[mgrp].append(cinfo)

# function to output the segments list that a particular muscle receives input from
def segments_for_muscle(muscle_name):
    segments = load_motoneurons.loc[load_motoneurons["target_muscle_name"]==muscle_name]['segments'].tolist()[0].split(",")
    #print segments
    return segments
def get_muscles_for_movement(movement_type):
    muscles_for_movement = []
    for each in muscles_roster['muscle_name']:
        each_df = muscles_roster.loc[muscles_roster['muscle_name']==each]
        movements = each_df['movement_type'].tolist()[0].split(",")
        for movement in movements:
            if movement == movement_type:
                muscles_for_movement.append(each)
    return muscles_for_movement
#get_muscles_for_movement("ankle_plantarflexion")
def get_antagonist_movement(movement_type):
    joint,movement = movement_type.split("_")
    antagonist_movement = joint +"_"+ movements_roster.loc[movements_roster['movement_type']==movement]['antagonist_movement'].tolist()[0]
    return antagonist_movement    
#get_antagonist_movement("ankle_plantarflexion")


def make_cell_group(neuron_type,target_muscle_name,segment,side,numcells,lamina,transmitter,hoc_template_name):
    #print segment
    #print target_muscle_name
    cell_group = {u'axonr': '',
                 u'axons': '',
                              u'cell_group_region': lamina+"_"+side,
                              u'cell_packing_adapter': json.dumps({u'number': str(numcells), u'pdf': u'random'}),
                              u'cell_type': neuron_type,
                              u'dends': '',
                              u'funcs': '',
                              u'name': lamina+"_"+side+"_"+neuron_type+"_"+target_muscle_name,
                              u'neuro_rx': '',
                              u'neuro_tx': transmitter,
                              u'sub_cell_type': target_muscle_name,
                              u'seg_info':segment,
                              u'side':side,
                              u'hoc_template':hoc_template_name}
    return cell_group

def create_moto_cellgroups(moto_roster):
    motoneuron_cellgroups = []
    for each in moto_roster:
        lamina = each['lamina']
        neuron_type = each['neuron_type']
        target_muscle_name = each['target_muscle_name']
        transmitter = each['transmitter']
        numcells = each['numcells']
        segments_list = each['segments'].split(",")
        spinal_side_list = []
        hoc_template_name = each['hoc_template']
        if each['side(L/R/both)'] == 'both':
            spinal_side_list = ['L','R']
        else:
            spinal_side_list = each['side(L/R/both)']
            
        for each_side in spinal_side_list:
            for each_segment in segments_list:
                motoneuron_cellgroups.append(make_cell_group(neuron_type,target_muscle_name,each_segment,each_side,numcells,lamina,transmitter,hoc_template_name))
    return(motoneuron_cellgroups)

def create_inter_cellgroups(inter_roster):
    interneuron_cellgroups = []
    for each in inter_roster:
        lamina = each['lamina']
        neuron_type = each['neuron_type']
        transmitter = each['transmitter']
        numcells = each['numcells']
        hoc_template_name = each['hoc_template']
        target_muscles_list = []
        segments_list =[]
        if (each['target_muscle_name'] == 'All') & (each['segments'] == 'mirror'):
            target_muscles_list = list(load_motoneurons['target_muscle_name'])
            #print 1
        else:
            target_muscles_list = each['target_muscle_name']
            segments_list = each['segments'].split(",")
            
        
        spinal_side_list = []
        if each['side(L/R/both)'] == 'both':
            spinal_side_list = ['L','R']
        else:
            spinal_side_list = each['side(L/R/both)']
            
        for each_side in spinal_side_list:
            for each_target_muscle_name in target_muscles_list:
                #print each_target_muscle_name
                segments_list = segments_for_muscle(each_target_muscle_name)
                #print segments_list
                for each_segment in segments_list:
                    interneuron_cellgroups.append(make_cell_group(neuron_type,each_target_muscle_name,each_segment,each_side,numcells,lamina,transmitter,hoc_template_name))
    return(interneuron_cellgroups)

def create_drg_cellgroups(drg_roster):
    drg_cellgroups = []
    for each in drg_roster:
        #lamina = each['lamina']
        neuron_type = each['neuron_type']
        transmitter = each['transmitter']
        numcells = each['numcells']
        hoc_template_name = each['hoc_template']
        target_muscles_list = []
        segments_list =[]
        if (each['target_muscle_name'] == 'All') & (each['segments'] == 'mirror'):
            target_muscles_list = list(load_motoneurons['target_muscle_name'])
            #print 1
        else:
            target_muscles_list = each['target_muscle_name']
            segments_list = each['segments'].split(",")
            
        
        spinal_side_list = []
        if each['side(L/R/both)'] == 'both':
            spinal_side_list = ['L','R']
        else:
            spinal_side_list = each['side(L/R/both)']
            
        for each_side in spinal_side_list:
            for each_target_muscle_name in target_muscles_list:
                #print each_target_muscle_name
                segments_list = segments_for_muscle(each_target_muscle_name)
                for each_segment in segments_list:
                    lamina = each_segment + "_" + each['lamina']
                    drg_cellgroups.append(make_cell_group(neuron_type,each_target_muscle_name,each_segment,each_side,numcells,lamina,transmitter,hoc_template_name))
    return(drg_cellgroups)

#caonnect takes input as cell_groups_dataframe, template_connection_rule, and connection_modality
#output the connections 
def connect(cell_groups_dataframe,template,connection_modality):
    #source_cell_groups = {}
    #target_cell_groups = {}
    ncs = []
    source_neuron = template['src']
    target_neuron = template['dest']
    connect_side = template['side']
    source_neuron_pool = cell_groups_dataframe.loc[(cell_groups_dataframe['cell_type']==source_neuron)]
    target_neuron_pool = cell_groups_dataframe.loc[(cell_groups_dataframe['cell_type']==target_neuron)]
    #print target_neuron_pool['sub_cell_type']
    source_sub_cell_types = set(cell_groups_dataframe['sub_cell_type'].tolist())
    #net_conn = None
    if connection_modality == 'self':
           for sub_cell_type in source_sub_cell_types:
                #print sub_cell_type
                #target_neuron_pool['sub_cell_type'] == sub_cell_type
                target_cell_groups = target_neuron_pool.loc[(target_neuron_pool['sub_cell_type']==sub_cell_type)]
                source_cell_groups = source_neuron_pool.loc[source_neuron_pool['sub_cell_type']==sub_cell_type]
                source_cell_groups = source_cell_groups.T.to_dict().values()
                target_cell_groups = target_cell_groups.T.to_dict().values()
                #print source_cell_groups
                #nc.append(connect_cell_groups(source_cell_groups,target_cell_groups,connect_side,template['props']))
                net_conn = connect_cell_groups(source_cell_groups,target_cell_groups,connect_side,template['props'])
                #print net_conn
                if net_conn != None:
                    for each in net_conn:
                        if each not in ncs:
                            ncs.append(each)
    if connection_modality == 'agonist':
        for sub_cell_type in source_sub_cell_types:
            #check what movements the sub_cell_type i.e agnistic to muscle_name involves in  
            movements = muscles_roster.loc[muscles_roster['muscle_name']==sub_cell_type]['movement_type'].tolist()[0].split(",")
            for each_movement_type in movements:
                agonist_muscles_list = get_muscles_for_movement(each_movement_type)
                #to make sure that current muscle group is removed so only gives agonist list of muscles
                agonist_muscles_list.remove(sub_cell_type)
                
                if agonist_muscles_list == None:
                    pass
                else:
                        for each_agonist_muscle in agonist_muscles_list:
                            target_cell_groups = target_neuron_pool.loc[target_neuron_pool['sub_cell_type']==each_agonist_muscle]
                            source_cell_groups = source_neuron_pool.loc[source_neuron_pool['sub_cell_type']==sub_cell_type]
                            source_cell_groups = source_cell_groups.T.to_dict().values()
                            target_cell_groups = target_cell_groups.T.to_dict().values()
                            #nc.append(connect_cell_groups(source_cell_groups,target_cell_groups,connect_side,template['props']))
                            net_conn = connect_cell_groups(source_cell_groups,target_cell_groups,connect_side,template['props'])
                            #print net_conn
                            if net_conn != None:
                                for each in net_conn:
                                    if each not in ncs:
                                        ncs.append(each)
    if connection_modality == 'antagonist':
        for sub_cell_type in source_sub_cell_types:
            #check what movementsn the sub_cell_type i.e agnistic to muscle_name involves in  
            movements = muscles_roster.loc[muscles_roster['muscle_name']==sub_cell_type]['movement_type'].tolist()[0].split(",")
            #print movements
            for each_movement_type in movements:
                antagonist_muscles_list = get_muscles_for_movement(get_antagonist_movement(each_movement_type))
                #print antagonist_muscles_list
                for each_antagonist_muscle in antagonist_muscles_list:
                    target_cell_groups = target_neuron_pool.loc[target_neuron_pool['sub_cell_type']==each_antagonist_muscle]
                    source_cell_groups = source_neuron_pool.loc[source_neuron_pool['sub_cell_type']==sub_cell_type]
                    source_cell_groups = source_cell_groups.T.to_dict().values()
                    target_cell_groups = target_cell_groups.T.to_dict().values()
                    #nc.append(connect_cell_groups(source_cell_groups,target_cell_groups,connect_side,template['props']))
                    net_conn = connect_cell_groups(source_cell_groups,target_cell_groups,connect_side,template['props'])
                    if net_conn != None:
                        for each in net_conn:
                            if each not in ncs:
                                ncs.append(each)
    return ncs
#connect the cell_groups takes input as source cell_pool, target_cell _pool, connect_side and props as input
def connect_cell_groups(source_cell_pool,target_cell_pool,connect_side,props):
    net_conn = []
    if connect_side == 'ipsilateral':
        #print source_cell_pool
        for each_source_cell_group in source_cell_pool:
            for each_target_cell_group in target_cell_pool:
                if each_source_cell_group['side'] == each_target_cell_group['side']:
                    net_con = make_connection(s_neuron = each_source_cell_group['cell_type'],
                                                          s_neuron_subtype = each_source_cell_group['sub_cell_type'],
                                                            s_neuron_lamina = each_source_cell_group['cell_group_region'],
                                                            s_neuron_seg = each_source_cell_group['seg_info'],
                                                            d_neuron = each_target_cell_group['cell_type'], 
                                                            d_neuron_subtype = each_target_cell_group['sub_cell_type'],
                                                            d_neuron_lamina = each_target_cell_group['cell_group_region'],
                                                            d_neuron_seg = each_target_cell_group['seg_info'],
                                                            transmitter = each_source_cell_group['neuro_tx'],
                                                            props = props)
                    net_conn.append(net_con)
    if connect_side == 'contralateral':
        for each_source_cell_group in source_cell_pool:
            for each_target_cell_group in target_cell_pool:
                if (each_source_cell_group['side'] == L) & (target_cell_group['side'] == R):
                    net_con = make_connection(s_neuron = each_source_cell_group['cell_type'],
                                                            s_neuron_subtype = each_source_cell_group['sub_cell_type'],
                                                            s_neuron_lamina = each_source_cell_group['cell_group_region'],
                                                            s_neuron_seg = each_source_cell_group['seg_info'],
                                                            d_neuron = each_target_cell_group['cell_type'], 
                                                            d_neuron_subtype = each_target_cell_group['sub_cell_type'],
                                                            d_neuron_lamina = each_target_cell_group['cell_group_region'],
                                                            d_neuron_seg = each_target_cell_group['seg_info'],
                                                            props = props)
                    net_conn = net_conn + net_con
                if (each_source_cell_group['side'] == R) & (target_cell_group['side'] == L):
                    net_con = make_connection(s_neuron = each_source_cell_group['cell_type'],
                                                            s_neuron_subtype = each_source_cell_group['sub_cell_type'],
                                                            s_neuron_lamina = each_source_cell_group['cell_group_region'],
                                                            s_neuron_seg = each_source_cell_group['seg_info'],
                                                            d_neuron = each_target_cell_group['cell_type'], 
                                                            d_neuron_subtype = each_target_cell_group['sub_cell_type'],
                                                            d_neuron_lamina = each_target_cell_group['cell_group_region'],
                                                            d_neuron_seg = each_target_cell_group['seg_info'],
                                                            props = props)
                    net_conn = net_conn + net_con
    return net_conn
#makes the net_connection and generates 
def make_connection(s_neuron,s_neuron_subtype,s_neuron_lamina,s_neuron_seg,d_neuron,d_neuron_subtype,d_neuron_lamina,d_neuron_seg,props,transmitter): 
    net_conn = { "source_segment": "Human_"+s_neuron_seg ,
                             "source_cellgroup": s_neuron_lamina+"_"+s_neuron+"_"+s_neuron_subtype,
                             "dest_segment": "Human_"+d_neuron_seg,
                             "dest_cellgroup": d_neuron_lamina+"_"+d_neuron+"_"+d_neuron_subtype,
                             "axon_route":'',
                             "nc_props": props,
                             "transmitter":transmitter,
                              }
    return net_conn


#write cell_groups to cell_groups.xlsx
writer1 = pd.ExcelWriter(get_cellgroups_file())
cell_groups_info = create_drg_cellgroups(neurons_roster['drg_neurons'])+create_moto_cellgroups(neurons_roster['motoneurons'])+create_inter_cellgroups(neurons_roster['interneurons'])
cellgroup_cols = ["name","cell_type","sub_cell_type","cell_group_region","axons","dends","cell_packing_adapter","axonr","funcs",            "neuro_tx","neuro_rx"]
tempdf = pd.DataFrame(cell_groups_info)
emptydf = pd.DataFrame()
emptydf.to_excel(writer1,sheet_name="common")
for each in set(tempdf['seg_info'].tolist()):
    seg_name="Human_" + each
    df = pd.DataFrame(tempdf.loc[tempdf['seg_info']==each])
    df.to_excel(writer1,sheet_name=seg_name,columns=cellgroup_cols,index=False)
writer1.save()
writer1.close()
#tempdf.loc[tempdf['seg_info']=='L4']

#write net_cons to nc_explicit.xlsx
total_conns = []
nc = []
writer2 = pd.ExcelWriter(get_nc_explicit_file())
netcon_cols = ["source_segment","source_cellgroup","dest_segment","dest_cellgroup","axon_route","nc_props","transmitter"]
for each_modality in connections_info.keys():
    for each_template in connections_info[each_modality]:
        connexions = (connect(tempdf,each_template,each_modality))
        #connect(tempdf,each_template,each_modality)
        nc.extend(connexions)
#         for each_conn in connexions:
#              if each_conn not in total_conns:
#                 total_conns.append(each_conn)
for each in nc:
    if each not in total_conns:
        total_conns.append(each)
nc_df = pd.DataFrame(total_conns)
nc_df.to_excel(writer2,sheet_name="common",columns=netcon_cols,index=False)
writer2.save()
writer2.close()

In [25]:
connections_info['agonist'][0]['props'] = eval(connections_info['agonist'][0]['props'])

In [34]:
connections_info['agonist'][0]['props']['weight'] = 0.5

In [35]:
connections_info['agonist'][0]['props']['weight'] 

0.5

In [46]:
json.dumps(connections_info)

'{"self": [{"dest": "aMot", "src": "Ia", "side": "ipsilateral", "props": {"delay": 5, "fan_in": 1, "weight": 0.7, "name": "Ia_Exci"}}, {"dest": "IaIn", "src": "Ia", "side": "ipsilateral", "props": {"delay": 3, "fan_in": 1, "weight": 0.5}}, {"dest": "IbIn", "src": "Ib", "side": "ipsilateral", "props": {"delay": 2, "fan_in": 1, "weight": 0.75}}, {"dest": "aMot", "src": "IbIn", "side": "ipsilateral", "props": {"delay": 2, "fan_in": 1, "weight": 0.5, "name": "Int_Inhi"}}, {"dest": "aMot", "src": "IbEx", "side": "ipsilateral", "props": {"delay": 3, "fan_in": 1, "weight": 0.5, "name": "Int_Exci"}}, {"dest": "Ren", "src": "aMot", "side": "ipsilateral", "props": {"delay": 4, "fan_in": 1, "weight": 0.5}}, {"dest": "aMot", "src": "Ren", "side": "ipsilateral", "props": {"delay": 5, "fan_in": 1, "weight": 0.5, "name": "Int_Inhi"}}, {"dest": "aMot", "src": "II", "side": "ipsilateral", "props": {"delay": 2, "fan_in": 1, "weight": 0.65, "name": "Ia_Exci"}}, {"dest": "IIint", "src": "II", "side": "ips

In [42]:
import copy
cfd = copy.deepcopy(connections_info)

In [36]:
cfd 

In [44]:
connections_info['agonist'][0]['props']['weight']

3

In [2]:
def muscles_for_seg(seg):
    muscles_ = []
    for each in neurons_roster['motoneurons']:
        if seg in each['segments']:
            muscles_.append(each['target_muscle_name'])
    return muscles_

In [3]:
seg_musc_dict = {}
for each in spinal_segments:
    seg_musc_dict[each]= muscles_for_seg(each)

NameError: name 'spinal_segments' is not defined

In [ ]:
#rearrage muscles as per the segmental order
muscles_seg_order = []
for each in spinal_segments:
    for each_musc in seg_musc_dict[each]:
        if each_musc not in muscles_seg_order:
            muscles_seg_order.append(each_musc)        

In [ ]:
set(seg_musc_dict['S1']) & set(seg_musc_dict['S2']) 

In [ ]:
set(seg_musc_dict['L4']) & set(seg_musc_dict['L5']) & set(seg_musc_dict['S1']) & set(seg_musc_dict['S2']) 

In [ ]:
set(seg_musc_dict['L5']) & set(seg_musc_dict['S1']) & set(seg_musc_dict['S2']) & set(seg_musc_dict['S3'])

In [ ]:
len(total_conns)

In [2]:
load_mus = pd.read_excel("muscles_for_segs1.xlsx")
muscles_seg_order = load_mus['muscles'].to_list()

In [ ]:
tempdf.loc[tempdf['cell_type']=='aMot']

In [ ]:
nc_df

In [8]:
#extract only the left ones from nc_df
left_con_df = nc_df[nc_df['dest_cellgroup'].str.contains("_L_") | nc_df['dest_cellgroup'].str.contains("_L_")]

In [9]:
left_con_df
left_con_df = left_con_df.reset_index()

In [5]:
#luboscaral rostral_to_caudal
import numpy as np
spinal_segments = np.array(["L1","L2","L3","L4","L5","S1","S2","S3","S4","S5"])
def segment_idx(segment):
    seg_idx = np.where(spinal_segments == segment)
    seg_idx = seg_idx[0][0]
    return float(seg_idx) 
def intersegmental_flow(src_segment,tar_segment):
    weight = 0
    if src_segment == tar_segment:
        weight = 2
        return weight
    else:
        weight = abs(segment_idx(src_segment)-segment_idx(tar_segment))
        return round(1.0/weight,2)

In [31]:
#segment rules
segs = {"L1":1,"L2":2,"L3":3,"L4":4,"L5":5,"S1":6,"S2":7,"S3":8}
def calc_seg_score(src_seg,tar_seg):
    seg_diff = segs[src_seg]-segs[tar_seg]
    seg_score = 0
    if seg_diff < 0:
        seg_score = 1.0/(-seg_diff)
    if seg_diff == 0:
        seg_score = 1.0
    if seg_diff > 1:
        seg_score = 1.0/seg_diff
    return seg_score

In [32]:
calc_seg_score('L2','L5')

0.3333333333333333

In [10]:
#extract source neurons
source_neuron_grp = list(map(lambda(i, j): i + "_" + j, zip(left_con_df["source_segment"].to_list(), left_con_df["source_cellgroup"].to_list())))
#ectract target neurons
target_neuron_grp = list(map(lambda(i, j): i + "_" + j, zip(left_con_df["dest_segment"].to_list(), left_con_df["dest_cellgroup"].to_list())))
#extract tranmitter information
transmitter = left_con_df["transmitter"].to_list()
intersegmental_inf = []
#transmitter Score
transmitter_score = []
for i in range(len(left_con_df)):
    if left_con_df.loc[i,"transmitter"] == 'GLU':
        score = 1
    else:
        score = -1
    transmitter_score.append(score)

#compute intersegmental influence
for i in range(len(left_con_df)):
    #<usage> icalc_seg_score(src_seg,tar_seg)
    new_num = calc_seg_score(left_con_df.loc[i,"source_segment"].split("_")[1],left_con_df.loc[i,"dest_segment"].split("_")[1])
    #print new_num
    intersegmental_inf.append(new_num)
    
#compute the connectivity scores
# for i in range(len(left_con_df))

In [ ]:
#left_con_df = left_con_df.reset_index()
left_con_df.loc[0,'dest_segment']

In [11]:
data = {'source_neuron_grp':source_neuron_grp,
       'target_neuron_grp':target_neuron_grp,
       'transmitter':transmitter,
       'intersegmental_inf':intersegmental_inf,
       'transmitter_score':transmitter_score }
new_data = pd.DataFrame(data, columns=['source_neuron_grp','target_neuron_grp','transmitter','transmitter_score',
                                       'intersegmental_inf'])

In [ ]:
filename = cwd + "/data.xlsx"
writer3 = pd.ExcelWriter(filename)
new_data.to_excel(writer3,sheet_name="common",index=False)
writer3.save()
writer3.close()

In [ ]:
new_data

In [ ]:
len(set(source_neuron_grp)) 

In [ ]:
len(set(target_neuron_grp))

In [ ]:
list(set(target_neuron_grp))[0]

In [ ]:
list(set(source_neuron_grp))[0]

In [9]:
#generate a zero matrix with size : source x target 
source_neuron_grp_set = list(set(source_neuron_grp))
target_neuron_grp_set = list(set(target_neuron_grp))
total_neuron_grp = source_neuron_grp_set + target_neuron_grp_set
total_neuron_grp_set = list(set(total_neuron_grp))
adj_mat_trans = pd.DataFrame(0, index = total_neuron_grp_set , columns = total_neuron_grp_set)
adj_mat_seg_infl = pd.DataFrame(0.0, index = total_neuron_grp_set , columns = total_neuron_grp_set)
#fill the matix with the tramitter influence and segmental influence
for i in range(len(new_data)):
    source_neuron = new_data.loc[i,'source_neuron_grp']
    target_neuron = new_data.loc[i,'target_neuron_grp']
    trans_score = new_data.loc[i,'transmitter_score']  
    seg_infl = new_data.loc[i,'intersegmental_inf']
    #print seg_infl
    adj_mat_trans.at[source_neuron,target_neuron] = trans_score
    adj_mat_seg_infl.at[source_neuron,target_neuron] = seg_infl

In [ ]:
len(total_neuron_grp_set)

In [ ]:
adj_mat_trans

In [ ]:
adj_mat_seg_infl.loc['Human_L4_L4_drg_L_Ia_bifemst','Human_S2_Hm9_L_aMot_bifemst']

In [ ]:
#write the matrix to xlsx
filename_adj_mat = cwd + "/adj_mat_trans.xlsx"
writer4 = pd.ExcelWriter(filename_adj_mat)
adj_mat_trans.to_excel(writer4,sheet_name="common",index=True)
writer4.save()
writer4.close()

In [ ]:
#write the matrix to xlsx
filename_adj_mat = cwd + "/adj_mat_seg_infl.xlsx"
writer5 = pd.ExcelWriter(filename_adj_mat)
adj_mat_seg_infl.to_excel(writer5,sheet_name="common",index=True)
writer5.save()
writer5.close()

In [ ]:
muscles=load_motoneurons['target_muscle_name'].to_list()

In [2]:
calc_seg_score("S3","L1")

0.14285714285714285

In [ ]:
for each in total_conns:
    if each['source_cellgroup'].split('_')[3] == 'Ia':
        target_celltype = each['dest_cellgroup'].split('_')[2]
        if target_celltype == 'IaIn':
            print each

In [12]:
def find_tar_neuron(src_neuron,total_conn):
    tar_neuron_list = []
    for each in total_conn:
        if each['source_cellgroup'] == src_neuron:
            tar_neuron_list.append(each['dest_segment']+"_"+each['dest_cellgroup'])
    return tar_neuron_list

In [ ]:
find_tar_neuron('8Sp_L_IbEx_glumed1',total_conns)

In [13]:
#code to generate the Ia interaction matrices based on individual spinal connections
#code to generate the muscle synergies matrix
import numpy as np

#insert required synergy to build for visualization
syn = "Ia"

#create an empty dataframe i.e an adjacency matrix with muscles names.
d1 = pd.DataFrame(0, index = muscles_seg_order, columns = muscles_seg_order)

#iterate over connection to find the desired source neuron
for each in total_conns:
    if each['source_cellgroup'].split('_')[3] == syn:
        source_muscle = each['source_cellgroup'].split('_')[4]
        dest_muscle = each['dest_cellgroup'].split('_')[3]
        target_celltype = each['dest_cellgroup'].split('_')[2]
        source_seg = each['source_cellgroup'].split('_')[0]
        tar_seg = each['dest_segment'].split('_')[1]
        #print tar_seg
        
        #monosynaptic based Ia scorring
        if target_celltype == "aMot":
            if source_muscle == dest_muscle:
                #print 'yes'
                #add score +1 to the 
                d1.at[source_muscle,dest_muscle]=d1.at[source_muscle,dest_muscle]+2*(calc_seg_score(source_seg,tar_seg))
                #print d1.at[source_muscle,dest_muscle]
            else:
                d1.at[source_muscle,dest_muscle]=d1.at[source_muscle,dest_muscle]+calc_seg_score(source_seg,tar_seg)

        #disynaptic structures
        if target_celltype == "IaIn": 
            #find the aMot targets of IaIn
            tar_list = find_tar_neuron(each['dest_cellgroup'],total_conns)
            for each in tar_list:
                if each.split("_")[4] == 'aMot':
                    dest_muscle = each.split("_")[5] 
                    tar_seg = each.split("_")[1]
                    #print each
                    if source_muscle != dest_muscle:
                        d1.at[source_muscle,dest_muscle]=d1.at[source_muscle,dest_muscle]-(calc_seg_score(source_seg,tar_seg))


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))
#title = "Ia synergy matrix"
#plt.title(title, fontsize = 20)

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(d1, cmap=cmap, center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 1})
#vmax=100,vmin=-100
plt.show()
f.savefig('Ia_interaction.png', bbox_inches='tight', pad_inches=0.1)

NameError: name 'muscles_seg_order' is not defined

In [ ]:
for each in total_conns:
    if each['source_cellgroup'].split('_')[3] == 'Ib':
        print each

In [ ]:
d1

In [ ]:
d1.at['gaslat','gaslat']

In [ ]:
d1.min().min()

In [ ]:
#code to generate the Ib interaction matrices based on individual spinal connections
#code to generate the muscle synergies matrix
import numpy as np

#insert required synergy to build for visualization
syn = "Ib"

#create an empty dataframe i.e an adjacency matrix with muscles names.
d2 = pd.DataFrame(0, index = muscles_seg_order, columns = muscles_seg_order)

#iterate over connection to find the desired source neuron
for each in total_conns:
    if each['source_cellgroup'].split('_')[3] == syn:
        source_muscle = each['source_cellgroup'].split('_')[4]
        dest_muscle = each['dest_cellgroup'].split('_')[3]
        target_celltype = each['dest_cellgroup'].split('_')[2]
        source_seg = each['source_cellgroup'].split('_')[0]
        tar_seg = each['dest_segment'].split('_')[1]
        #print tar_seg

        #disynaptic structures
        if target_celltype == "IbEx": 
            #find the aMot targets of IaIn
            tar_list = find_tar_neuron(each['dest_cellgroup'],total_conns)
            #print tar_list
            for each in tar_list:
                if each.split("_")[4] == 'aMot':
                    dest_muscle = each.split("_")[5] 
                    tar_seg = each.split("_")[1]
                    #print source_muscle, dest_muscle
#                     if source_muscle == dest_muscle:
#                         d2.at[source_muscle,dest_muscle]=d1.at[source_muscle,dest_muscle]+(-1)-calc_seg_score(source_seg,tar_seg)
                    if source_muscle != dest_muscle:
                        d2.at[source_muscle,dest_muscle]=d2.at[source_muscle,dest_muscle]+calc_seg_score(source_seg,tar_seg)
                        #print source_muscle, dest_muscle, d2.at[source_muscle,dest_muscle]
        #disynaptic structures
        if target_celltype == "IbIn": 
            #find the aMot targets of IaIn
            tar_list = find_tar_neuron(each['dest_cellgroup'],total_conns)
            for each in tar_list:
                if each.split("_")[4] == 'aMot':
                    dest_muscle = each.split("_")[5] 
                    tar_seg = each.split("_")[1]
                    #print each
                    if source_muscle == dest_muscle:
                        d2.at[source_muscle,dest_muscle]=d2.at[source_muscle,dest_muscle]-calc_seg_score(source_seg,tar_seg)
#                     if source_muscle != dest_muscle:
#                         d2.at[source_muscle,dest_muscle]=d1.at[source_muscle,dest_muscle]+1+calc_seg_score(source_seg,tar_seg)


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))
#title = "Ib synergy matrix"
#plt.title(title, fontsize = 20)

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(d2, cmap=cmap, center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 1})
plt.show()
f.savefig('Ib_interaction.png', bbox_inches='tight', pad_inches=0.1)

In [ ]:
sns.clustermap(d1)

In [ ]:
#code to generate the renshaw interation matrices of the individual spinal connections
#code to generate the muscle synergies matrix
import numpy as np
#load total circuit stack 
#total_connections = moto_run(total_muscles)
#create an empty dataframe i.e an adjacency matrix with muscles names.
d3 = pd.DataFrame(0, index = muscles, columns = muscles)
for each in total_conns:
    if each['source_cellgroup'].split('_')[2] == 'aMot':
        source_muscle = each['source_cellgroup'].split('_')[3]
        dest_muscle = each['dest_cellgroup'].split('_')[3]
        target_celltype = each['dest_cellgroup'].split('_')[2]
        
        #find aMot targets for each
        if target_celltype == "Ren":
            tar_list = find_tar_neuron(each['dest_cellgroup'],total_conns)
            for each in tar_list:
                if each.split("_")[4] == 'aMot':
                    dest_muscle = each.split("_")[5] 
                    tar_seg = each.split("_")[1]
                    #print each
                    if source_muscle == dest_muscle:
                        d3.at[source_muscle,dest_muscle]=d3.at[source_muscle,dest_muscle]+2*((-1)-calc_seg_score(source_seg,tar_seg)) 
                    if source_muscle != dest_muscle:
                        d3.at[source_muscle,dest_muscle]=d3.at[source_muscle,dest_muscle]+(-1)-calc_seg_score(source_seg,tar_seg)

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))
#title = "Renshaw synergy matrix"
#plt.title(title, fontsize = 20)

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

#mask to draw only lower triangle
mask = np.zeros_like(d3)
mask[np.triu_indices_from(mask,k=1)] = True
# Draw the heatmap with the mask and correct aspect rati0
sns.heatmap(d3, cmap=cmap,center=0, mask=False,annot=False,
            square=False, linewidths=1, cbar_kws={"shrink": 1})
#vmin =-200, vmax=0,
plt.show()
f.savefig('Ren_interaction.png', bbox_inches='tight', pad_inches=0.1)

In [ ]:
#build synapse matrix

#####Rules######
#scores
#calss: Ia-aMot -> monosynaptic
## self -> 2*seg_score
## agonist -> 1*seg_score

#class: Ia-IaIn-aMot - disynaptic
## Ia-IaIn -> 1*seg_score 
## IaIn-aMot -> -2*seg_score - strong antagonist
## IaIn-aMot -> -1*seg_score - weak antagonist

#class: Ib-IbIn-aMot - disyanaptic
## Ib-IbIn -> 

import numpy as np

#load spine.json here and examine the netcons
#path to original spine.json generated in neuroid
path_to_json = 'spine_01_05_20.json'
with open(path_to_json) as f:
    spine_all = json.load(f)
    
spine_all_ = spine_all.copy()
    
#store the 'net_connection' and 'cell_groups' to the temporary     
net_conns = spine_all_['net_connections']
cgrps = spine_all_['cell_groups']

##connection details to dataframe for easy look up and modification

#create a data structure for source, target and conection parameters parameters 
#also attach connection detils to each connection


#list of all types of the cells
neuron_list = ['Ia','II','Ib','IaIn','IbEx','IbIn','Ren','aMot']
sesory_list = ['Ia','II','Ib']
inter_neuron_list = ['IaIn','IbEx','IbIn','Ren']
moto_neuron_list = ['aMot']

#empty lists 
conn_neu_type_list = []
conn_mus_type_list = []
conn_seg_flow_list = []
conn_dict = []


for each_conn in net_conns:
    
    #pool the details
    source_cell = each_conn['sregion']
    target_cell = each_conn['tregion']
    target_cell_type = each_conn['destination']['cell_type'].split("_")[0]
    source_cell_type = each_conn['source']['cell_type'].split("_")[0]
    delay = each_conn['synapse']['delay']
    weight = each_conn['synapse']['weight']
    divergence = np.nan
    
    #extract info from the string
    src_var = source_cell.split("_")
    tar_var = target_cell.split("_")
    src_mus = src_var[len(src_var)-1]
    tar_mus = tar_var[len(tar_var)-1]
    src_seg = src_var[1]
    tar_seg = tar_var[1]
    
    #find the neuron type for source and target cell
    for each_neuron in neuron_list: 
        tar_strng_idx = target_cell.find(each_neuron)
        src_strng_idx = source_cell.find(each_neuron)
        if tar_strng_idx != -1:
            tar_cell_type = each_neuron
        if src_strng_idx != -1:
            src_cell_type = each_neuron
            
    #construct the connection type
    conn_neu_type = src_cell_type + "-" + tar_cell_type
    conn_mus_type = src_mus + "-" + tar_mus
    seg_flow = src_seg + "-" + tar_seg
    
    #determine the synapse type
    if (source_cell_type == "Ia") or (source_cell_type == "Ib") or (source_cell_type == "II") or (source_cell_type == "IbEx"):
        syn_type = "excitatory"
    else:
        syn_type = "inhibitory"
    
    #excetptional case which will override the previous rule
    if (target_cell_type == "aMot") and (source_cell_type == "Ren"):
        syn_type = 'inhibitory'
        
    #append all the details
    conn_dict.append({'source_cell': source_cell,
                     'target_cell' : target_cell,
                     'conn_type_neu' : conn_neu_type,
                     'conn_type_mus' : conn_mus_type,
                     'seg_flow' : seg_flow,
                     'delay' : delay,
                     'weight' : weight,
                     'divergence' : divergence,
                     'syn_type' : syn_type})
    
#convert connect info to dataframe    
conn_df = pd.DataFrame(conn_dict,columns=['source_cell','target_cell','conn_type_neu','conn_type_mus','seg_flow','delay','weight','divergence','syn_type'] )

In [10]:
# build synapse matrix
syn_dict = []
for each in total_conns:
    #print each
    src_cell = each['source_cellgroup']
    tar_cell = each['dest_cellgroup']
    tar_cell_type = tar_cell.split("_")[2]
    
    #print src_cell, tar_cell
    
    if (src_cell.split("_")[3] == 'Ia'):  #[ monosynaptic connections ]
        #print 'yes'
        syn_type = "excitatory"
        syn_sign = 2 
        src_cell_type = src_cell.split("_")[3]
        
    elif (src_cell.split("_")[3] == 'Ib') or (src_cell.split("_")[3] == 'II') or (src_cell.split("_")[3] == 'IbEx'):
            syn_type = "excitatory"
            syn_sign = 1
            src_cell_type = src_cell.split("_")[3]
        
    elif (src_cell.split("_")[2] == 'Ren') and (tar_cell.split("_")[2] == 'aMot'):  
            syn_type = "inhibitory"
            syn_sign = -1
            src_cell_type = src_cell.split("_")[2]
            
    elif (src_cell.split("_")[2] == 'IaIn') and (tar_cell.split("_")[2] == 'IaIn'):  
        syn_type = "inhibitory"
        syn_sign = -1
        src_cell_type = src_cell.split("_")[2]
        
    else:
          src_cell_type =  src_cell.split("_")[2]
          print src_cell_type
          syn_type = "inhibitory"
          syn_sign = -1

    segScore = calc_seg_score(each['source_segment'].split("_")[1],each['dest_segment'].split("_")[1])
    con_type = src_cell_type + "-" + tar_cell_type
    
    
    syn_val = syn_sign*segScore
    
    syn_dict.append({'source_cell': src_cell,
                     'target_cell': tar_cell,
                     'con_type': con_type,
                     'syn_val': syn_val,
                     'syn_type': syn_type,
                     'seg_flow':each['source_segment'].split("_")[1]+"-"+each['dest_segment'].split("_")[1]})

IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn
IbIn


IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn
IaIn


In [11]:
syn_df = pd.DataFrame(syn_dict, columns =['source_cell','target_cell','seg_flow','syn_val','syn_type','con_type'])

In [47]:
syn_df[syn_df['con_type']=='Ia-aMot']

,source_cell,target_cell,seg_flow,syn_val,syn_type,con_type
0,L4_drg_L_Ia_bifemst,Hm9_L_aMot_bifemst,L4-L4,2.000000,excitatory,Ia-aMot
1,L4_drg_L_Ia_bifemst,Hm9_L_aMot_bifemst,L4-L5,2.000000,excitatory,Ia-aMot
2,L4_drg_L_Ia_bifemst,Hm9_L_aMot_bifemst,L4-S1,1.000000,excitatory,Ia-aMot
3,L4_drg_L_Ia_bifemst,Hm9_L_aMot_bifemst,L4-S2,0.666667,excitatory,Ia-aMot
4,L5_drg_L_Ia_bifemst,Hm9_L_aMot_bifemst,L5-L4,0.000000,excitatory,Ia-aMot
5,L5_drg_L_Ia_bifemst,Hm9_L_aMot_bifemst,L5-L5,2.000000,excitatory,Ia-aMot
6,L5_drg_L_Ia_bifemst,Hm9_L_aMot_bifemst,L5-S1,2.000000,excitatory,Ia-aMot
7,L5_drg_L_Ia_bifemst,Hm9_L_aMot_bifemst,L5-S2,1.000000,excitatory,Ia-aMot
8,S1_drg_L_Ia_bifemst,Hm9_L_aMot_bifemst,S1-L4,1.000000,excitatory,Ia-aMot
9,S1_drg_L_Ia_bifemst,Hm9_L_aMot_bifemst,S1-L5,0.000000,excitatory,Ia-aMot


In [61]:
syn_df[syn_df['con_type']=='Ib-aMot']

,source_cell,target_cell,seg_flow,syn_val,syn_type,con_type
876,8Sp_L_IbIn_bifemst,Hm9_L_aMot_bifemst,S2-L4,-0.333333,inhibitory,Ib-aMot
877,8Sp_L_IbIn_bifemst,Hm9_L_aMot_bifemst,S2-L5,-0.500000,inhibitory,Ib-aMot
878,8Sp_L_IbIn_bifemst,Hm9_L_aMot_bifemst,S2-S1,0.000000,inhibitory,Ib-aMot
879,8Sp_L_IbIn_bifemst,Hm9_L_aMot_bifemst,S2-S2,-1.000000,inhibitory,Ib-aMot
880,8Sp_L_IbIn_bifemst,Hm9_L_aMot_bifemst,L4-L4,-1.000000,inhibitory,Ib-aMot
881,8Sp_L_IbIn_bifemst,Hm9_L_aMot_bifemst,L4-L5,-1.000000,inhibitory,Ib-aMot
882,8Sp_L_IbIn_bifemst,Hm9_L_aMot_bifemst,L4-S1,-0.500000,inhibitory,Ib-aMot
883,8Sp_L_IbIn_bifemst,Hm9_L_aMot_bifemst,L4-S2,-0.333333,inhibitory,Ib-aMot
884,8Sp_L_IbIn_bifemst,Hm9_L_aMot_bifemst,L5-L4,0.000000,inhibitory,Ib-aMot
885,8Sp_L_IbIn_bifemst,Hm9_L_aMot_bifemst,L5-L5,-1.000000,inhibitory,Ib-aMot


In [13]:
total_conns[0]

{'axon_route': '',
 'dest_cellgroup': u'Hm9_L_aMot_bifemst',
 'dest_segment': u'Human_L4',
 'nc_props': u'{"weight": 0.7, "delay": 5, "fan_in":1, "name": "Ia_Exci"}',
 'source_cellgroup': u'L4_drg_L_Ia_bifemst',
 'source_segment': u'Human_L4',
 'transmitter': u'GLU'}

In [18]:
#load major and minor movers information here

#create a dataframe with 
muscle_movements_parse = pd.read_excel(get_muscles_input_file(),sheetname='Sheet4',index_col=0)
muscle_movements_dataframe = pd.DataFrame(0, index = muscles_seg_order, columns = muscle_movements_parse.index.tolist())
for each_movement in muscle_movements_dataframe.columns:
    #print each_movement
    for each_muscle in muscle_movements_dataframe[each_movement].index:
        #print each_muscle
        #print each_movement
        prime_movers = muscle_movements_parse['prime_movers'].loc[each_movement]
        accessory_movers = muscle_movements_parse['accessory_movers'].loc[each_movement]
        if prime_movers != 'nil':
            if each_muscle in muscle_movements_parse['prime_movers'].loc[each_movement].split(','):
                muscle_movements_dataframe[each_movement].loc[each_muscle] = 2
        if accessory_movers != 'nil':
             if each_muscle in muscle_movements_parse['accessory_movers'].loc[each_movement].split(','):
                muscle_movements_dataframe[each_movement].loc[each_muscle] = 1 

In [ ]:
muscle_movements_dataframe

In [17]:
#construct movement_profiles

#from the total neurons list extract the motoneurons list:
moto_set = []
for each in total_neuron_grp_set:
    if each.split("_")[4] == 'aMot':
        moto_set.append(each)

In [ ]:
#extract motoneurons of desired segment
def moto_for_seg(seg_name,moto_set):
    moto_seg_list =[]
    for each in moto_set:
        if each.split("_")[1] == seg_name:
            moto_seg_list.append(each)
    return moto_seg_list

In [22]:
#motoneuron activity with one
moto_act_df = pd.DataFrame(0,index = moto_set, columns=['act'])
moto_act_df.loc["Human_L3_Ad9_L_aMot_pec"]

act    0
Name: Human_L3_Ad9_L_aMot_pec, dtype: int64

In [ ]:
moto_act_df.at["Human_L3_Ad9_L_aMot_pec",'act']

In [20]:
#set_moto_activity
def set_activity(moto_set,seg_name,act = 1):
    #create empty dataframe with index mot names and activty column
    moto_act_df_temp = pd.DataFrame(0,index = moto_set, columns=[seg_name])
    for each in moto_act_df.index.to_list():
        #print each
        if each.split("_")[1] == seg_name:
            #print "yes"
            moto_act_df_temp.at[each,seg_name] = act
    return moto_act_df_temp

In [ ]:
set_activity(moto_set,"L4",act = 1)

In [23]:
#create the dataframe with all the activity 
total_act_df = pd.DataFrame()
for each_seg in segs.keys():
    act_df = set_activity(moto_set,each_seg,act=1)
    #print each_seg,act_df[act_df>0] 
    total_act_df = pd.concat([total_act_df,act_df],axis=1)
total_act_df = total_act_df.sort_index(axis=1)

In [ ]:
total_act_df.sort_index(axis=1)

In [24]:
#construct muscle activity dataframe from motoneuronal activity dataframe(fictive not dynamic)
muscle_act_df = pd.DataFrame(0,index = muscles_seg_order, columns=segs.keys())
for each_seg in segs.keys():
    #extract the moto activity of each segment and add the value to the muscle
    #temp_df = total_act_df[each]:
    for each_moto in total_act_df.index.tolist():
        moto_act = total_act_df.loc[each_moto,each_seg]
        mus_name = each_moto.split("_")[5]
        muscle_act_df.at[mus_name,each_seg]=muscle_act_df.at[mus_name,each_seg]+moto_act

In [110]:
muscle_act_df['L4'][muscle_act_df['L4']==1].index.tolist()

[u'addmgdis',
 u'addmgis',
 u'addmgmid',
 u'recf',
 u'vaslat',
 u'vasmed',
 u'vasint',
 u'ta',
 u'tp',
 u'semmem',
 u'glumin1',
 u'glumin2',
 u'glumin3',
 u'glumed1',
 u'glumed2',
 u'glumed3',
 u'bifemlg',
 u'bifemst',
 u'semtend']

In [26]:
movements_set = muscle_movements_parse.index.tolist()

In [27]:
#open-loop
muscle_movements_exp = pd.DataFrame(0, index =  movements_set, columns = segs.keys())
muscle_movements_exp = muscle_movements_exp.sort_index(axis=1)
temp_dict_ol = []
for each_seg in muscle_act_df.columns.values.tolist():
    print each_seg
    #extract the active muscles data from the muscle activation profile data
    active_muscles_list= muscle_act_df[each_seg][muscle_act_df[each_seg]>0].index.tolist()
    temp_motion_types = []
    #print each_exp
    for each_muscle in active_muscles_list:
        #print each_muscle
        #find motion_type the muscle involved
        motion_types = muscles_roster[muscles_roster['muscle_name'] == each_muscle]['movement_type'].values[0].split(',')
        #print each_muscle, motion_types
        
        for each_element in motion_types:
            #find scores for rach motion_type caused by that muscle and append to the dataframe
            print each_element
            if each_element in movements_set:
                movement_score = muscle_movements_dataframe[each_element].loc[each_muscle]
                # add the movement score to the dataframe
                muscle_movements_exp[each_seg].loc[each_element]= muscle_movements_exp[each_seg].loc[each_element] + movement_score
                temp_motion_types.append(each_element)
    set_motion_types=[]
    for each in temp_motion_types:
        if each not in set_motion_types:
            set_motion_types.append(each)
    temp_dict_ol.append({each_seg:set_motion_types})

L4
hip_adduction
hip_internalRotation
hip_adduction
hip_internalRotation
hip_adduction
hip_internalRotation
knee_extension
hip_flexion
knee_extension
knee_extension
knee_extension
ankle_dorsiflexion
ankle_inversion
ankle_plantarflexion
ankle_inversion
knee_flexion
hip_extension
hip_internalRotation
hip_abduction
hip_internalRotation
hip_abduction
hip_internalRotation
hip_abduction
hip_internalRotation
hip_abduction
hip_internalRotation
hip_abduction
hip_internalRotation
hip_abduction
hip_internalRotation
knee_flexion
hip_extension
hip_externalRotation
knee_flexion
hip_extension
knee_flexion
hip_extension
hip_internalRotation
L5
ankle_plantarflexion
ankle_inversion
knee_flexion
hip_extension
hip_internalRotation
hip_abduction
hip_internalRotation
hip_abduction
hip_internalRotation
hip_abduction
hip_internalRotation
hip_abduction
hip_internalRotation
hip_abduction
hip_internalRotation
hip_abduction
hip_internalRotation
knee_flexion
hip_extension
hip_externalRotation
knee_flexion
hip_exte

In [ ]:
['knee_flexion','knee_extension'] in movements_set

In [ ]:
muscles_roster[muscles_roster['muscle_name'] == 'addmn']['movement_type'].values[0].split(',')

In [ ]:
muscle_act_df.index.tolist() == muscles_roster['muscle_name'].values.tolist()

In [ ]:
muscles_roster['muscle_name'].values.tolist()

In [36]:
#plotting code 
import pygal
import cairosvg
pygal.style.Style.legend_font_size=18
#pygal.config.Config.legend_box_size = 12
#pygal.style.Style.major_label_font_size=40
#pygal.style.Style.value_label_font_size=40
pygal.style.Style.label_font_size = 25
pygal.style.Style.title_font_size = 30
dot_chart = pygal.Dot(x_label_rotation=50,width=1000,height=800,legend_at_bottom=True, legend_box_size=18)
dot_chart.title = 'movement_profile'
dot_chart.x_labels = muscle_movements_exp.index
#dot_chart.y_labels  = 'L2_lam9_inj','L3_lam9_inj','L4_lam9_inj','L5_lam9_inj','S1_lam9_inj','S2_lam9_inj','S3_lam9_inj'
dot_chart.y_labels  = 'L1','L2','L3','L4','L5','S1','S2','S3'
for each_exp in muscle_movements_exp.columns:
    dot_chart.add(each_exp, muscle_movements_exp[each_exp].values)
dot_chart.render_to_png("movements_profile1.png")
dot_chart.render_in_browser()

file:///tmp/tmpLjL_Az.html


In [41]:
muscle_movements_exp.to_excel("seg_movements.xlsx")

In [35]:
#close-loop

#We have movements from open loop 

# For any given movement, the antagonist muscles stretch and causes Ia responses to trigger:
# so for each segment pool antagonist movements:

muscle_movements_exp['L1'][muscle_movements_exp['L1']>0].index.tolist()

[u'hip_flexion', u'knee_flexion']

In [ ]:
get_antagonist_movement

In [ ]:
muscle_movements_exp

In [ ]:
moto_for_seg("L3",moto_set)

In [ ]:
segs.keys()

In [ ]:
#plot the movement profile
def 

In [ ]:
moto_for_seg

In [ ]:
cell_template_map = []
for each in cell_groups_info:
    temp_grp = {'cell_type':each['cell_type'],
                'sub_cell_type':each['sub_cell_type'],
                'template': each['hoc_template']}
    if temp_grp not in cell_template_map:
        cell_template_map.append(temp_grp)
        
template_map_df = pd.DataFrame(cell_template_map)
writer3 = pd.ExcelWriter("cell_template_map.xlsx")
template_map_df.to_excel(writer3,sheet_name="map",index=False)
writer3.save()
writer3.close()

In [ ]:
import os
import pandas as pd
cwd = os.getcwd()
#print cwd
cell_groups_file = pd.ExcelFile(cwd + "/cell_groups.xlsx")
nc_template = pd.read_excel(cwd + "/nc_explicit.xlsx")
cell_groups_file.sheet_names
sheet_to_df_map = {}

for sheet_name in cell_groups_file.sheet_names:
    #print sheet_name
    sheet_to_df_map[sheet_name] = cell_groups_file.parse(sheet_name)
    
cell_groups_master = []
for each_df in sheet_to_df_map:
    if each_df != "common":
        cell_groups_master.extend(map(lambda cell_grp_name: each_df +"_"+ cell_grp_name, sheet_to_df_map[each_df]["name"].to_list()))

In [ ]:
source_neuron_grp = list(map(lambda(i, j): i + "_" + j, zip(nc_template["source_segment"].to_list(), nc_template["source_cellgroup"].to_list())))
target_neuron_grp = list(map(lambda(i, j): i + "_" + j, zip(nc_template["dest_segment"].to_list(), nc_template["dest_cellgroup"].to_list())))

In [ ]:
for i in range(0,len(source_neuron_grp)):
    print source_neuron_grp[i], "--->", target_neuron_grp[i], i

In [ ]:
#code to plot the neuroid_osim maps and osim_neuroid maps
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


neuroid_osim_map = pd.read_excel("moto_osim_musc_map.xlsx")
neuroid_osim_map['seg_influence'] = 1 

osim_muscles = set(list(neuroid_osim_map['muscle_name'].values.tolist()))
motoneurons = set(list(neuroid_osim_map['source_motoneuron'].values.tolist()))

empty_d = pd.DataFrame(0, index = osim_muscles, columns = motoneurons)

for each_motoneuron in motoneurons:
    target_muscle_name = neuroid_osim_map[neuroid_osim_map["source_motoneuron"] == each_motoneuron]['muscle_name'].values
    #print target_muscle_name
    seg_influence = neuroid_osim_map[neuroid_osim_map["source_motoneuron"] == each_motoneuron]['seg_influence'].values
    empty_d.at[target_muscle_name,each_motoneuron] = seg_influence
    
empty_d = empty_d[sorted(empty_d.columns)]
    
%matplotlib inline
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(110, 90))
title = "neuroid_osim_map"
plt.title(title, fontsize = 20)
# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(empty_d.transpose(), cmap=cmap, vmin =0, vmax=1, center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": 1}, cbar=False, ax=ax)
plt.show()
f.savefig(title + '.pdf', bbox_inches='tight', pad_inches=0.1, dpi=2000)

In [ ]:
empty_d

In [ ]:
empty_d[sorted(empty_d.columns)]

In [ ]:
neuroid_osim_map[neuroid_osim_map["source_motoneuron"] == 'Human_L5_GasSol9_L_aMot_gaslat']['seg_influence'][0]

In [ ]:
import pandas as pd
import os
import numpy as np
#cluster analysis
#load the clusters from the xlsx sheet

cluster_dict = {}
for i in range(0,20):
    #cluster_id = "cluster" + "_" + i
    cluster = pd.read_excel("../testdata.xlsx",sheet_name=i)
    #print i
    cluster_dict["cluster_"+str(i+1)] = cluster 

In [ ]:
cluster_dict["cluster_20"]

In [ ]:
for each in cluster_dict:
    print cluster_dict[cluster_1]

In [ ]:
neuron_types = ["Ia","Ib","II","aMot","IaIn","IbEx","IbIn","Ren"]

In [ ]:
cluster_dict[0]["cluster_1"]['Var1']

In [ ]:
#neuron_counter = {"Ia":[],"Ib":[],"II":[],"aMot":[],"IaIn":[],"IbEx":[],"IbIn":[],"Ren":[]}
#identify neuron types
muscles_list = []
cluster_details ={}
for each_cluster in cluster_dict:
    #list of label splits
    neuron_counter = {"Ia":[],"Ib":[],"II":[],"aMot":[],"IaIn":[],"IbEx":[],"IbIn":[],"Ren":[]}
    segments_counter = {"L1":[],"L2":[],"L3":[],"L4":[],"L5":[],"S1":[],"S2":[],"S3":[]}
    muscles_counter = []
    #segment_cpunts = []
    cluster = cluster_dict[each_cluster]
    #print each_cluster
    neuron_labels = cluster["Var1"]
    #extract details from the labels
    for each_label in neuron_labels:
        label_split = each_label.split("_")
        
        #print each_label, label_split
        neuron_found = set(label_split) & set(neuron_types)
        neuron = list(neuron_found) #only one neuron type to be identified
        #create a dict of neurons:
        
        #last item after splitting the string is muscle name
        target_muscle = label_split[len(label_split)-1]
        muscles_counter.append(target_muscle)
        
        if target_muscle not in muscles_list:
            muscles_list.append(target_muscle)
            
        neuron_counter[neuron[0]].append(neuron[0])
        #return lengths of the list of each neuron and append them into a dict
        segments_counter[label_split[1]].append(label_split[1])
        
    neuron_counts = {}
    for each in neuron_counter:
        neuron_counts[each] = len(neuron_counter[each])
        
    segment_counts = {} 
    for each in segments_counter:
        segment_counts[each] = len(segments_counter[each])
    
        
    cluster_details[each_cluster] = {"neuron_counts":neuron_counts, 
                                     "total_neuron_count":len(neuron_labels),
                                     "segment_counts": segment_counts,
                                     "muscles": muscles_counter}

In [ ]:
clutser_musc_details = {}
for each in cluster_details:
    cluster_musc = cluster_details[each]['muscles'] 
    temp_dict = {}
    for each_musc in muscles_list:
        muscle_counts = cluster_musc.count(each_musc)
        temp_dict[each_musc] = muscle_counts
    clutser_musc_details[each] = {'muscles':temp_dict}
clutser_musc_details

In [ ]:
#functin to sort alphaneurmeric strings
import re
def alphaNumOrder(string):
   """ Returns all numbers on 5 digits to let sort the string with numeric order.
   Ex: alphaNumOrder("a6b12.125")  ==> "a00006b00012.00125"
   """
   return ''.join([format(int(x), '05d') if x.isdigit()
                   else x for x in re.split(r'(\d+)', string)])

In [ ]:
cluster_keys = cluster_details.keys()
cluster_keys.sort(key=alphaNumOrder)
cluster_id = map(lambda x: int(x.split("_")[1]), cluster_details.keys())

In [ ]:
cluster_details.keys()
data_neurons = []
data_segs = []
data_muscles = []
data_neuron_num = []
for each in cluster_details:
    data_neurons.append(cluster_details[each]['neuron_counts'])
    data_segs.append(cluster_details[each]['segment_counts'])
    data_neuron_num.append(cluster_details[each]['total_neuron_count'])
    #print cluster_details[each]['neuron_counts']
    data_muscles.append(clutser_musc_details[each]['muscles'])

In [ ]:
data_neuron_num

In [ ]:
data_muscles

In [ ]:
#clusters vs neurons vs number of neurons
import matplotlib.pyplot as plt
x_vars = cluster_details.keys()
y_vars = neuron_types

In [ ]:
#clusters and neuron_types
df1 = pd.DataFrame(data_neurons, index=cluster_details.keys())
df1['cluster_id'] = cluster_id
df1.sort_values('cluster_id',inplace=True)
df1.drop('cluster_id',axis=1,inplace=True)

#clusters and segments
df2 = pd.DataFrame(data_segs,index=cluster_details.keys())
df2['cluster_id'] = cluster_id
df2.sort_values('cluster_id',inplace=True)
df2.drop('cluster_id',axis=1,inplace=True)

#clusters and muscles
df3 = pd.DataFrame(data_muscles,index=cluster_details.keys())
df3['cluster_id'] = cluster_id
df3.sort_values('cluster_id',inplace=True)
df3.drop('cluster_id',axis=1,inplace=True)

#clusters and number of neurons
df4 = pd.DataFrame(data_neuron_num,index=cluster_details.keys(), columns=['number of neurons'])
df4['cluster_id'] = cluster_id
df4.sort_values('cluster_id',inplace=True)
df4.drop('cluster_id',axis=1,inplace=True)

In [ ]:
dataframe_list = [df1,df2,df3,df4]
dataframe_names = ["cluster_vs_neuron_types","cluster_vs_segments","cluster_vs_muscles","cluster_vs_num_neurons"]
ite = 0
for each in dataframe_list:
    writer = pd.ExcelWriter(cwd + "/" + dataframe_names[ite] +".xlsx")
    each.to_excel(writer,sheet_name="common")
    writer.save()
    writer.close()
    ite += 1

In [ ]:
each

In [ ]:
#df4.rename(columns={"0": "number of neurons"})
df4.plot.bar()

In [ ]:
pd.DataFrame.from_dict(data_neurons, index = )

In [ ]:
dfu1 = df1.unstack().reset_index()
dfu2 = df2.unstack().reset_index()
dfu3 = df3.unstack().reset_index()
dfu1.columns = list("XYS")
dfu2.columns = list("XYS")
dfu3.columns = list("XYS")
dfu1["S"] *= 10
dfu2["S"] *= 10
dfu3["S"] *= 30

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
plt.scatter(x="Y", y="X", s="S", data=dfu1)
#plt.xticks()[1].set_fontsize(10)
plt.tick_params(labelsize=20)
plt.xticks(rotation=70)
#plt.tick_params(axis='X', labelrotation = 20)
#plt.rc('axes', labelsize=40)
#plt.margins(0.05)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
#dfu2["S"] *= 10
plt.figure(figsize=(20,10))
plt.scatter(x="Y", y="X", s="S", data=dfu2)
plt.tick_params(labelsize=20)
plt.xticks(rotation=70)
#plt.margins(0.05)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
#dfu2["S"] *= 10
plt.figure(figsize=(20,30))
plt.scatter(x="Y", y="X", s="S", data=dfu3)
plt.tick_params(labelsize=20)
plt.xticks(rotation=70)
#plt.margins(0.05)
plt.show()

In [ ]:
seg